In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=513c2d89344b5df6c1d8719db8072e7cc141cd76c337041ee3a9a60759d0ba26
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('hack').getOrCreate()
df=spark.read.csv('hack_data.csv',inferSchema=True,header=True)

In [3]:
df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [4]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
df.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [7]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [13]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors

In [9]:
df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [10]:
assembler=VectorAssembler(inputCols=['Session_Connection_Time','Bytes Transferred','Kali_Trace_Used',
                     'Servers_Corrupted','Pages_Corrupted','WPM_Typing_Speed'],outputCol='features')

In [11]:
output=assembler.transform(df)

In [12]:
output.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
from pyspark.ml.feature import StandardScaler

In [16]:
scale=StandardScaler(inputCol='features',outputCol='scaledFeatures')

In [17]:
final_data=scale.fit(output).transform(output)

In [46]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



In [14]:
from pyspark.ml.clustering import KMeans

In [31]:
kmeans2=KMeans(featuresCol='scaledFeatures',k=2)
kmeans3=KMeans(featuresCol='scaledFeatures',k=3)

In [32]:
model2=kmeans2.fit(final_data)
model3=kmeans3.fit(final_data)

In [34]:
#WSSSE(Within Set of Sum of Squared Errors)
wssse2=model2.summary.trainingCost
wssse3=model3.summary.trainingCost

In [43]:
wssse2

601.7707512676691

In [44]:
wssse3

434.75507308487596

In [47]:
for k in range(2,9):
  kmeans=KMeans(featuresCol='scaledFeatures',k=k)
  model=kmeans.fit(final_data)
  wssse=model.summary.trainingCost
  print('With k={}'.format(k))
  print('Within Set Sum of Squared Errors = '+str(wssse))
  print('--'*30)

With k=2
Within Set Sum of Squared Errors = 601.7707512676691
------------------------------------------------------------
With k=3
Within Set Sum of Squared Errors = 434.75507308487596
------------------------------------------------------------
With k=4
Within Set Sum of Squared Errors = 267.1336116887894
------------------------------------------------------------
With k=5
Within Set Sum of Squared Errors = 252.95948530127137
------------------------------------------------------------
With k=6
Within Set Sum of Squared Errors = 233.6273127670655
------------------------------------------------------------
With k=7
Within Set Sum of Squared Errors = 210.00049537843455
------------------------------------------------------------
With k=8
Within Set Sum of Squared Errors = 208.38947578501254
------------------------------------------------------------


In [37]:
centers2=model2.clusterCenters()
centers3=model3.clusterCenters()

In [38]:
centers2

[array([2.99991988, 2.92319035, 1.05261534, 3.20390443, 4.51321315,
        3.28474   ]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612])]

In [39]:
centers3

[array([3.05623261, 2.95754486, 1.99757683, 3.2079628 , 4.49941976,
        3.26738378]),
 array([1.26023837, 1.31829808, 0.99280765, 1.36491885, 2.5625043 ,
        5.26676612]),
 array([2.93719177, 2.88492202, 0.        , 3.19938371, 4.52857793,
        3.30407351])]

In [48]:
model3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   79|
|         0|   88|
+----------+-----+



In [49]:
model2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

